# Implementation of SegNet on Drone Images 

This notebook recreates the procedure of data preperation and data loading and also initializing, training and evaluating SegNet on this data.

As the code is organized in modules, large portions of the produced code is not contained directly herein, but can be consulted by opening the `.py` files in the `src/`-folder.

NOTE: Because the full network takes upwards of 30 GB of memory to train and evaluate, a simplified version is used here. The full version can be accessed by setting the `use_simple` variable to `False`. Runtime evaluation is also disabled by default to save memory. It is explained how to enable it further down.

We recommend running this notebook on Google Colab.

In [1]:
!echo "Make sure that you are in /src before running this notebook"
!pwd  # Prints current path (Linux and MacOS only)

# For running on google colab
# %cd /content
# !git clone https://github.com/sorenmulli/alpha_cellari_imageseg.git
# %cd alpha_cellari_imageseg/src
# %mkdir logs
# %pwd

"Make sure that you are in /src before running this notebook"
'pwd' is not recognized as an internal or external command,
operable program or batch file.


## Data preperation

Downloads and prepares data. This will take a number of minutes to run and requires a little over a gigabyte of storage.

In [2]:
!python -m pip install wget --user
!python data_prepper.py

2020-01-03 20:04:42.989258	Preparation of data with images of shape (512, 512, 3)

2020-01-03 20:04:42.990221	Downloading images...
2020-01-03 20:04:52.757543	Done downloading images to paths
                          	local_data/raw.png
                          	local_data/target.png

2020-01-03 20:04:52.758541	Loading images...
2020-01-03 20:04:55.065867	Done loading images
                          	Shapes: (6815, 5364, 3)
                          	Split: (0.83, 0.17)

2020-01-03 20:04:55.066870	Padding images...
2020-01-03 20:04:56.075968	Done padding images
                          	Shapes: (7168, 5632, 3)

2020-01-03 20:04:56.086904	Saving subimages to location local_data/imgs/{type}-{index}.png
2020-01-03 20:05:15.751248	Done saving subimages

2020-01-03 20:05:15.752246	Standardizing aerial image...
2020-01-03 20:05:15.755237	Done standardizing image

2020-01-03 20:05:15.755237	Squeezing target images to single channel...
2020-01-03 20:09:12.988029	Done creating target values

## Preparing network training


In [3]:
from logger import Logger
from train import Trainer
from model_test import Tester
from augment import Augmenter, AugmentationConfig, flip_lr, flip_tb
import torch

print("Configuring parameters...")

# If True, a simpler version of segnet is used
use_simple = True

# Configuration of network architechture
architecture = {
		"kernel_size":  3,
		"padding": 1, 
		"stride": 1,
		"pool_dims": (2, 2),
		"probs": 0.1,
		"reduce_complexity": use_simple,
}

learning_rate = 1.5e-4
batch_size = 3
epochs = 3000

# Configuration of data augmentation
# Reducing cropsize significantly reduces memory usage and training time
augmentations = AugmentationConfig(
    augments =  [flip_lr, flip_tb],  
    cropsize = (350, 350),
    augment_p = [0.5, 0.5]
)

torch.cuda.empty_cache()
logger = Logger("logs/train_run.log", "Running full training loop")

print("Done configuring")

Configuring parameters...
2020-01-03 20:10:11.252408	Running full training loop

Done configuring


## Training the Network

The following code trains the network.

Because we had access to significant compute recourses, we did not optimize the loss calculations for memory. It is therefore not possible to run evaluation on most machines, so it is disabled by default. It can be enabled by setting the `with_loss` argument to `True`. The loss curve is available in the report.

In [4]:
trainer = Trainer("local_data/prep_out.json", logger)
net = trainer.model_trainer(architecture, learning_rate, augmentations, epochs, batch_size, val_every = 25, with_loss = False, save_every = 500)
net.save('model')

2020-01-03 20:10:17.950306	Augmentations: {'cropsize': (350, 350), 'augments': [<function flip_lr at 0x000001C2D4CBED38>, <function flip_tb at 0x000001C2D3400CA8>], 'augment_p': [0.5, 0.5]}
2020-01-03 20:10:17.954296	Criterion and optimizer: CrossEntropyLoss()
                          	Adam (
                          	Parameter Group 0
                          	    amsgrad: False
                          	    betas: (0.9, 0.999)
                          	    eps: 1e-08
                          	    lr: 0.00015
                          	    weight_decay: 0
                          	)
2020-01-03 20:10:18.579215	Train size: 69
                          	Eval size: 15
                          	Test size: 24

2020-01-03 20:10:18.583206	Neural network information
                          		Net(
                          	  (encoder1): EncoderBlock(
                          	    (encoder): Sequential(
                          	      (0): BlueLayer(
                          	     

KeyboardInterrupt: 


## Evaluating the Network

In [0]:
from model import Net
# Load model or use existing
# net = Net.from_model("model")
tester = Tester("local_data/prep_out.json", logger)
tester.test_model(net, "local_data/test")


## Reconstructing the image

In the following, a complete forwarding is carried out using oversampling to prevent borders at the croppings.

In [0]:
from forward_passer import full_forward
full_forward(net, None, True, "local_data/full-forward.png")